## VFE Method and VRE Method
In this notebook, we estimate the alternative models which use variety fixed effects or variety random effects.

The output of this model are used in 60_Method_Compare_Plots

In [1]:
lapply(c('MASS',
         'stargazer',
         "lfe",
         "ggplot2",
         "dplyr",
         "strucchange",
         "gap",
         "boot",
         "stringr",
         "lme4"), require, character.only = TRUE)


df2 <- read.csv("../data/output_data/MG2_Private.csv")
df3 <- read.csv("../data/output_data/MG3_Private.csv")

DF2_CC <- df2 %>%                              # Applying group_by & summarise
  group_by(year) %>%
  summarise(count = n_distinct(company_name))

DF3_CC <- df3 %>%                              # Applying group_by & summarise
  group_by(year) %>%
  summarise(count = n_distinct(company_name))

df2 <- merge(df2,DF2_CC,by="year",all.x=T)
df3 <- merge(df3,DF3_CC,by="year",all.x=T)

df2$il_and_first <- as.numeric(df2$first_il=="IL_and_FIRST")
df3$il_and_first <- as.numeric(df3$first_il=="IL_and_FIRST")

df2_complete <- df2[complete.cases(df2$diff_d)&complete.cases(df2$diff_j),]
df2_reg23 <- df2_complete[(df2_complete$region=="region2")|(df2_complete$region=="region3"),]
df2_reg3 <- df2_complete[(df2_complete$region=="region3"),]
df3_complete <- df3[complete.cases(df3$diff_w),]
df3_reg23 <- df3_complete[(df3_complete$region=="region2")|(df3_complete$region=="region3"),]
df3_reg3 <- df3_complete[(df3_complete$region=="region3"),]

Y = c("diff_j_norm","diff_d_norm","diff_w_norm")

rain =  c('ppt_may', 'ppt_june', 'ppt_july', 'ppt_aug', 'ppt_sept',
          'ppt_may_sqr','ppt_june_sqr','ppt_july_sqr','ppt_aug_sqr','ppt_sept_sqr')

temp    =  c('tmin_5','tmin_6','tmin_7','tmin_8','tmin_9',
             'tmax_5','tmax_6','tmax_7','tmax_8','tmax_9',
             'tmean_5','tmean_6','tmean_7','tmean_8','tmean_9')

var_covars = c("height","lodging_comb","scn_resist","seed_treated")

factors = c("location","maturity_week","planting_week")

covars_w = c("height","lodging_comb","height_w","lodging_comb_w","seed_treated","scn_resist")
factors_w = c("location","maturity_week","maturity_week_w","planting_week")

covars_j = c("height","lodging_comb","height_j","lodging_comb_j","seed_treated","scn_resist")
factors_j = c("location","maturity_week","maturity_week_j","planting_week")

covars_d = c("height","lodging_comb","height_d","lodging_comb_d","seed_treated","scn_resist")
factors_d = c("location","maturity_week","maturity_week_d","planting_week")

fe_formula <- function(outcome,treat,X=var_covars,G=factors,C="location") {
  f <- paste0(outcome,"~",treat,"+",paste(X,collapse="+"),"|",paste(G,collapse="+"),"|","0","|",C)
  return(as.formula(f))
}
lm_formula <- function(outcome,treat,X=var_covars) {
  f <- paste0(outcome,"~",treat,"+",paste(X,collapse="+"))
  return(as.formula(f))
}

df2 <- read.csv("../data/output_data/MG2_Complete_FE.csv")
df3 <- read.csv("../data/output_data/MG3_Complete_FE.csv")

years_tested <- read.csv("../data/output_data/Years_Tested.csv")
names(years_tested) <- c("variety_id","years_tested")
variety_years2 <- df2[,c("variety_id","first_year")]
variety_years2 <- variety_years2[!duplicated(variety_years2),]

variety_years3 <- df3[,c("variety_id","first_year")]
variety_years3 <- variety_years3[!duplicated(variety_years3),]

var_covars = c("height","lodging_comb","scn_resist","seed_treated")
covars = c(var_covars,temp,rain)
factors = c("location","maturity_week","planting_week")


Loading required package: MASS

Loading required package: stargazer


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Loading required package: lfe

Loading required package: Matrix

Loading required package: ggplot2

Loading required package: dplyr


Attaching package: 'dplyr'


The following object is masked from 'package:MASS':

    select


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: strucchange

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: sandwich

Loading required package: gap

Loading required package: gap.datasets

gap version 1.6

Loading required package: boot

Loadi

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

## Estimators without check yield control

In [2]:
MFE <- felm(data=df2,formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-JACK')",
                                                   X=c(var_covars,rain,temp),G=factors))
MFE_ests <- summary(MFE)$coefficients
MFE1_coefs <- as.data.frame(MFE_ests[grep("variety_id", names(MFE_ests[,"Estimate"])),])
MFE1_names <- gsub(row.names(MFE1_coefs), pattern="relevel\\(factor\\(variety_id\\), \"public_variety-JACK\"\\)(.*?)",replacement="")
MFE1_coefs$variety_id <- MFE1_names
MFE1_coefs <- MFE1_coefs[,c("Estimate","variety_id")]
names(MFE1_coefs) <- c("VFE_nocheck","variety_id")

MFE1_coefs <- merge(MFE1_coefs,variety_years2)
MFE1_coefs <- merge(MFE1_coefs,years_tested)
MFE1_year2j <- lm(data =MFE1_coefs, VFE_nocheck ~ first_year)
MFE1_yearfe2j <- lm(data =MFE1_coefs, VFE_nocheck ~  relevel(as.factor(first_year),'1997'))
write.csv(summary(MFE1_yearfe2j)$coefficients,"../results/mfe1_year_j.csv")



MFE <- felm(data=df2,formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-DWIGHT')",
                                                   X=c(var_covars,rain,temp),G=factors))
MFE_ests <- summary(MFE)$coefficients
MFE1_coefs <- as.data.frame(MFE_ests[grep("variety_id", names(MFE_ests[,"Estimate"])),])
MFE1_names <- gsub(row.names(MFE1_coefs), pattern="relevel\\(factor\\(variety_id\\), \"public_variety-DWIGHT\"\\)(.*?)",replacement="")
MFE1_coefs$variety_id <- MFE1_names
MFE1_coefs <- MFE1_coefs[,c("Estimate","variety_id")]
names(MFE1_coefs) <- c("VFE_nocheck","variety_id")

MFE1_coefs <- merge(MFE1_coefs,variety_years2)
MFE1_coefs <- merge(MFE1_coefs,years_tested)
MFE1_year2d <- lm(data =MFE1_coefs, VFE_nocheck ~ first_year)
MFE1_yearfe2d <- lm(data =MFE1_coefs, VFE_nocheck ~  relevel(as.factor(first_year),'1997'))
write.csv(summary(MFE1_yearfe2d)$coefficients,"../results/mfe1_year_d.csv")


MFE <- felm(data=df3,formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-WILLIAMS_82')",
                                          X=c(var_covars,rain,temp),G=factors))
MFE_ests <- summary(MFE)$coefficients
MFE1_coefs <- as.data.frame(MFE_ests[grep("variety_id", names(MFE_ests[,"Estimate"])),])
MFE1_names <- gsub(row.names(MFE1_coefs), pattern="relevel\\(factor\\(variety_id\\), \"public_variety-WILLIAMS_82\"\\)(.*?)",replacement="")

MFE1_coefs$variety_id <- MFE1_names
MFE1_coefs <- MFE1_coefs[,c("Estimate","variety_id")]
names(MFE1_coefs) <- c("VFE_nocheck","variety_id")

MFE1_coefs <- merge(MFE1_coefs,variety_years3)
MFE1_coefs <- merge(MFE1_coefs,years_tested)
MFE1_year3w <- lm(data =MFE1_coefs, VFE_nocheck ~ first_year)
MFE1_yearfe3w <- lm(data =MFE1_coefs, VFE_nocheck ~  relevel(as.factor(first_year),'1997'))
write.csv(summary(MFE1_yearfe3w)$coefficients,"../results/mfe1_year_w.csv")


MRE1 <- lmer(lm_formula("yield_kg","(1|variety_id)",
                       X=c(c(var_covars,rain,temp),"as.factor(location)","as.factor(maturity_week)","as.factor(planting_week)")),
            data=df2)
MRE1_coefs <- coefficients(MRE1)
MRE1_coefs <- data.frame(VRE_nocheck = MRE1_coefs$variety_id[,"(Intercept)"],
                         variety_id=row.names(MRE1_coefs$variety_id))
MRE1_coefs <- merge(MRE1_coefs,variety_years2)
MRE1_coefs <- merge(MRE1_coefs,years_tested)
MRE1_year2 <- lm(data =MRE1_coefs, VRE_nocheck ~ first_year)
MRE1_yearfe2 <- lm(data =MRE1_coefs, VRE_nocheck ~  relevel(as.factor(first_year),'1997'))
write.csv(summary(MRE1_yearfe2)$coefficients,"../results/mre1_year_2.csv")


MRE1 <- lmer(lm_formula("yield_kg","(1|variety_id)",
                        X=c(c(var_covars,rain,temp),"as.factor(location)","as.factor(maturity_week)","as.factor(planting_week)")),
             data=df3)
MRE1_coefs <- coefficients(MRE1)
MRE1_coefs <- data.frame(VRE_nocheck = MRE1_coefs$variety_id[,"(Intercept)"],
                         variety_id=row.names(MRE1_coefs$variety_id))
MRE1_coefs <- merge(MRE1_coefs,variety_years3)
MRE1_coefs <- merge(MRE1_coefs,years_tested)
MRE1_year3 <- lm(data =MRE1_coefs, VRE_nocheck ~ first_year)
MRE1_yearfe3 <- lm(data =MRE1_coefs, VRE_nocheck ~  relevel(as.factor(first_year),'1997'))
write.csv(summary(MRE1_yearfe3)$coefficients,"../results/mre1_year_3.csv")


Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.0137813 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$ch

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'object' in selecting a method for function 'summary': object 'MRE1_yearfe' not found


### Estimators with check yield control

In [7]:
M0 <- felm(data=df2_complete,
           formula = fe_formula("diff_kg_j","",X=c(covars_j,rain,temp),G=factors_j))
M0_df <- model.frame(M0)
M0_df <- df2_complete[row.names(M0_df),]
M0_df$residuals <- M0$residuals
M0_coefs <- M0_df %>% group_by(variety_id) %>% summarize(Diff_Avg = mean(residuals))

M0_y2 <- felm(data=df2_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp),G=factors))

M0_y2j <- felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,rain,temp),G=factors_j))
M0_fe2j <- felm(data=df2_complete,formula = fe_formula("diff_kg_j","relevel(as.factor(first_year),'1997')",X=c(covars_j,rain,temp),G=factors_j))
M1_y2j  <- felm(data=df2_complete,formula = fe_formula("yield_kg","first_year + yield_kg_j",X=c(covars_j,rain,temp),G=factors_j))
M1_fe2j  <- felm(data=df2_complete,formula = fe_formula("yield_kg","relevel(as.factor(first_year),'1997') + yield_kg_j",X=c(covars_j,rain,temp),G=factors_j))

M0_y2d <- felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,rain,temp),G=factors_d))
M0_fe2d <- felm(data=df2_complete,formula = fe_formula("diff_kg_d","relevel(as.factor(first_year),'1997')",X=c(covars_d,rain,temp),G=factors_d))
M1_y2d  <- felm(data=df2_complete,formula = fe_formula("yield_kg","first_year + yield_kg_d",X=c(covars_d,rain,temp),G=factors_d))
M1_fe2d  <- felm(data=df2_complete,formula = fe_formula("yield_kg","relevel(as.factor(first_year),'1997') + yield_kg_d",X=c(covars_d,rain,temp),G=factors_d))

M0_y3 <- felm(data=df3_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp),G=factors))

M0_y3w <- felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,rain,temp),G=factors_w))
M0_fe3w <- felm(data=df3_complete,formula = fe_formula("diff_kg_w","relevel(as.factor(first_year),'1997')",X=c(covars_w,rain,temp),G=factors_w))
M1_y3w  <- felm(data=df3_complete,formula = fe_formula("yield_kg","first_year + yield_kg_w",X=c(covars_w,rain,temp),G=factors_w))
M1_fe3w  <- felm(data=df3_complete,formula = fe_formula("yield_kg","relevel(as.factor(first_year),'1997') + yield_kg_w",X=c(covars_w,rain,temp),G=factors_w))

stargazer(M0_y2,M0_y2j,MFE1_year2j,
          keep=c("year"), 
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Unadjusted","Yield","Differenced Yield","Yield","Differenced Yield","Yield"),
          out="../results/Model_Comparison_Table_MG2j.tex")

stargazer(M0_y2d,MFE1_year2d,
          keep=c("year"), 
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Unadjusted","Yield","Differenced Yield","Yield","Differenced Yield","Yield"),
          out="../results/Model_Comparison_Table_MG2d.tex")

stargazer(M0_y3,M0_y3w,MFE1_year3w,
          keep=c("year"), 
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Unadjusted","Yield","Differenced Yield","Yield","Differenced Yield","Yield"),
          out="../results/Model_Comparison_Table_MG3.tex")

stargazer(M0_y2j,M1_y2j,M0_y2d,M1_y2d,M0_y3w,M1_y3w,
          keep=c("year",'yield'), 
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend","Check Variety Yield"),
          dep.var.labels=c("Differenced Yield","Yield","Differenced Yield","Yield","Differenced Yield","Yield"),
          out="../results/Varying_Coef_Table.tex")

write.csv(summary(M1_fe2j)$coefficients,"../results/m1_year_j.csv")
write.csv(summary(M1_fe3w)$coefficients,"../results/m1_year_w.csv")



MFE2 <- felm(data=df2,formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-JACK') + yield_kg_j",
                                           X=c(covars_j,rain,temp),G=factors_j))
MFE2_ests <- summary(MFE2)$coefficients
MFE2_coefs <- as.data.frame(MFE2_ests[grep("variety_id", names(MFE2_ests[,"Estimate"])),])
MFE2_names <-  gsub(row.names(MFE2_coefs), pattern="relevel\\(factor\\(variety_id\\), \"public_variety-JACK\"\\)(.*?)",replacement="")
MFE2_coefs$variety_id <- MFE2_names
MFE2_coefs <- MFE2_coefs[,c("Estimate","variety_id")]
names(MFE2_coefs) <- c("VFE_check","variety_id")
MFE2_coefs <- merge(MFE2_coefs,variety_years2)
MFE2_coefs <- merge(MFE2_coefs,years_tested)
MFE2_yearj <- lm(data =MFE2_coefs, VFE_check ~ first_year)
MFE2_year_modj <- lm(data =MFE2_coefs[MFE2_coefs$first_year<2019,], VFE_check ~ first_year)
MFE2_yearfej <- lm(data =MFE2_coefs, VFE_check ~  relevel(as.factor(first_year),'1997'))
write.csv(summary(MFE2_yearfej)$coefficients,"../results/mfe2_year_j.csv")


MFE2 <- felm(data=df3,formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-WILLIAMS_82') + yield_kg_w",
                                           X=c(covars_w,rain,temp),G=factors_w))
MFE2_ests <- summary(MFE2)$coefficients
MFE2_coefs <- as.data.frame(MFE2_ests[grep("variety_id", names(MFE2_ests[,"Estimate"])),])
MFE2_names <- gsub(row.names(MFE2_coefs), pattern="relevel\\(factor\\(variety_id\\), \"public_variety-WILLIAMS_82\"\\)(.*?)",replacement="")
MFE2_coefs$variety_id <- MFE2_names
MFE2_coefs <- MFE2_coefs[,c("Estimate","variety_id")]
names(MFE2_coefs) <- c("VFE_check","variety_id")
MFE2_coefs <- merge(MFE2_coefs,variety_years3)
MFE2_coefs <- merge(MFE2_coefs,years_tested)
MFE2_yearw <- lm(data =MFE2_coefs, VFE_check ~ first_year)
MFE2_year_modw <- lm(data =MFE2_coefs[MFE2_coefs$first_year<2019,], VFE_check ~ first_year)

MFE2_yearfew <- lm(data =MFE2_coefs, VFE_check ~  relevel(as.factor(first_year),'1997'))
write.csv(summary(MFE2_yearfew)$coefficients,"../results/mfe2_year_w.csv")


MRE2 <- lmer(lm_formula("yield_kg","yield_kg_j + (1|variety_id)",
                X=c(c(covars_j,rain,temp),"as.factor(location)","as.factor(maturity_week)","as.factor(maturity_week_j)","as.factor(planting_week)")),
                data=df2)
MRE2_coefs <- coefficients(MRE2)
MRE2_coefs <- data.frame(VRE_check = MRE2_coefs$variety_id[,"(Intercept)"],variety_id=row.names(MRE2_coefs$variety_id))
MRE2_coefs <- merge(MRE2_coefs,variety_years3)
MRE2_coefs <- merge(MRE2_coefs,years_tested)
MRE2_year2 <- lm(data =MRE2_coefs, VRE_check ~ first_year)
MRE2_yearfe2 <- lm(data =MRE2_coefs, VRE_check ~  relevel(as.factor(first_year),'1997'))

write.csv(summary(MRE2_yearfe2)$coefficients,"../results/mre2_year_j.csv")

MRE2 <- lmer(lm_formula("yield_kg","yield_kg_w + (1|variety_id)",
                        X=c(c(covars_w,rain,temp),"as.factor(location)","as.factor(maturity_week)","as.factor(maturity_week_w)","as.factor(planting_week)")),
             data=df3)
MRE2_coefs <- coefficients(MRE2)
MRE2_coefs <- data.frame(VRE_check = MRE2_coefs$variety_id[,"(Intercept)"],variety_id=row.names(MRE2_coefs$variety_id))
MRE2_coefs <- merge(MRE2_coefs,variety_years3)
MRE2_coefs <- merge(MRE2_coefs,years_tested)
MRE2_year3 <- lm(data =MRE2_coefs, VRE_check ~ first_year)
MRE2_yearfe3 <- lm(data =MRE2_coefs, VRE_check ~  relevel(as.factor(first_year),'1997'))
write.csv(summary(MRE2_yearfe3)$coefficients,"../results/mre2_year_w.csv")


All_Coefs <- merge(merge(merge(merge(MRE2_coefs,MFE2_coefs),MFE1_coefs),MRE1_coefs),M0_coefs)
write.csv(All_Coefs,"../results/Robustness_Coefs.csv")

stargazer(M1_y2j,MFE2_yearj,MFE2_year_modj,MRE2_year2,
          keep=c("year","yield"),
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend","Check Variety Yield"),
          dep.var.labels=c("Unadj.","Unadj., Controls", "Adj. Jack", "Adj. Dwight"),
          out="../results/robustness_trends.tex")



% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Dec 11, 2024 - 11:57:27 AM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{3}{c}{\textit{Dependent variable:}} \\ 
\cline{2-4} 
\\[-1.8ex] & Unadjusted & Yield & Differenced Yield \\ 
\\[-1.8ex] & \textit{felm} & \textit{felm} & \textit{OLS} \\ 
\\[-1.8ex] & (1) & (2) & (3)\\ 
\hline \\[-1.8ex] 
 Year Trend & 29.735$^{***}$ & 25.225$^{***}$ & 31.424$^{***}$ \\ 
  & (7.936) & (3.534) & (0.989) \\ 
  & & & \\ 
\hline \\[-1.8ex] 
Observations & 23,207 & 23,207 & 2,814 \\ 
Adjusted R$^{2}$ & 0.534 & 0.449 & 0.264 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: mare

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.105731 (tol = 0.002, component 1)"


ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'object' in selecting a method for function 'summary': object 'MRE2_yearfe' not found
